<a href="https://colab.research.google.com/github/DEVjspf/ceu_facturas_tfa/blob/main/dev_facturas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto: Extracción de datos de facturas eléctricas
## Objetivo:
Extraer y gestionar información clave de facturas eléctricas para optimizar la asesoría y gestión mensual.

In [1]:
# Instalación de librerías necesarias
#!pip install langchain langchain-community langchain-experimental ollama PyMuPDF pydantic sqlalchemy psycopg[binary] faiss-cpu


### 📘 1. Importación de librerías necesarias

In [2]:
import os
import fitz  # PyMuPDF
import re
import json
from typing import List
from pydantic import BaseModel
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score, recall_score
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import requests

In [12]:
os.environ["TOGETHER_API_KEY"] = "tgp_v1_PHdIPUdwUiD4Br7N3HjeZbxeNJuEdTRpW9LrfkFWZe8"

### 📄 2. Cargar y leer texto desde el PDF

In [3]:
from pathlib import Path

import fitz  # PyMuPDF

ruta_facturas = Path("data/facturas/")

def extraer_texto_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf:
        texto_factura = ""
        for pagina in pdf:
            texto_factura += pagina.get_text()
    return texto_factura

for pdf_file in ruta_facturas.glob("*.pdf"):
    texto_factura = extraer_texto_pdf(pdf_file)
    print(f"\n📄 Archivo: {pdf_file.name}\n{'-'*40}")
    print(texto_factura[:500])
    print("-"*40)


📄 Archivo: Endesa Factura 08022024.pdf
----------------------------------------
VX10C02P-D-06/02/24 N0026249LNNNN
LARS TANKMAR TANKMAR
AVENIDA ONZE DE SETEMBRE 1 ESC-3 ATC-2
17255 BEGUR
GIRONA
Endesa Energía, S.A. Unipersonal. Inscrita en el Registro Mercantil de Madrid. Tomo 12.797, Libro 0, Folio 208,
 Sección 8ª, Hoja M-205.381, CIF A81948077. Domicilio Social: C/Ribera del Loira, nº60 28042 - Madrid.
Endesa Energía, S.A. Unipersonal.
CIF A81948077.
C/Ribera del Loira, nº 60 28042 - Madrid.
DATOS DE LA FACTURA
Nº factura: PMM401N0380480
Referencia: 012294855257/0412
Fech
----------------------------------------


### 🧾 3. Definir el esquema de los datos con Pydantic

In [4]:
from pydantic import BaseModel
from datetime import datetime

class FacturaElectrica(BaseModel):
    numero_factura: str
    fecha_emision: datetime
    periodo_inicio: datetime
    periodo_fin: datetime
    consumo_total_kwh: float
    potencia_punta_kw: float
    potencia_valle_kw: float
    importe_total: float


In [5]:
esquema_json = FacturaElectrica.schema_json(indent=2)
print(esquema_json)

{
  "properties": {
    "numero_factura": {
      "title": "Numero Factura",
      "type": "string"
    },
    "fecha_emision": {
      "format": "date-time",
      "title": "Fecha Emision",
      "type": "string"
    },
    "periodo_inicio": {
      "format": "date-time",
      "title": "Periodo Inicio",
      "type": "string"
    },
    "periodo_fin": {
      "format": "date-time",
      "title": "Periodo Fin",
      "type": "string"
    },
    "consumo_total_kwh": {
      "title": "Consumo Total Kwh",
      "type": "number"
    },
    "potencia_punta_kw": {
      "title": "Potencia Punta Kw",
      "type": "number"
    },
    "potencia_valle_kw": {
      "title": "Potencia Valle Kw",
      "type": "number"
    },
    "importe_total": {
      "title": "Importe Total",
      "type": "number"
    }
  },
  "required": [
    "numero_factura",
    "fecha_emision",
    "periodo_inicio",
    "periodo_fin",
    "consumo_total_kwh",
    "potencia_punta_kw",
    "potencia_valle_kw",
    "impor

<ipython-input-5-21c62e08e257>:1: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  esquema_json = FacturaElectrica.schema_json(indent=2)


### 🧩 5. Fragmentación semántica y generación de embeddings

In [6]:
import re
from typing import List
from sentence_transformers import SentenceTransformer

# 📌 Cargar modelo de embeddings
modelo_embeddings = SentenceTransformer("all-MiniLM-L6-v2")

# Nueva función para fragmentar texto por secciones específicas
def chunk_por_secciones(texto: str, secciones: List[str]) -> List[str]:
    # Crear patrón regex que detecte los títulos de secciones
    patron = '|'.join([re.escape(sec) for sec in secciones])
    indices = [m.start() for m in re.finditer(patron, texto)]
    indices.append(len(texto))  # Para el último chunk

    chunks = []
    for i in range(len(indices)-1):
        chunk = texto[indices[i]:indices[i+1]].strip()
        if chunk:
            chunks.append(chunk)
    return chunks

# Lista de secciones típicas en factura eléctrica
secciones = [
    "Datos de la factura",
    "Número de factura",
    "Fecha de emisión",
    "Periodo de facturación",
    "Consumo eléctrico",
    "Consumo total",
    "Potencia contratada",
    "Total"
]

# Fragmentar el texto de la factura por secciones
fragmentos = chunk_por_secciones(texto_factura, secciones)

# 📌 Generar embeddings a partir de los fragmentos obtenidos
embeddings = modelo_embeddings.encode(fragmentos, convert_to_tensor=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 🧩 5. Guardar embeddings en ChromaDB

In [7]:
!pip install chromadb

In [8]:
import chromadb

# Inicializar el cliente de ChromaDB (almacenamiento local por defecto)
client = chromadb.Client()

# Crear o obtener una colección
coleccion = client.get_or_create_collection("facturas_electricas")

# Guardar los embeddings y fragmentos en la colección
ids = [f"fragmento_{i}" for i in range(len(fragmentos))]
coleccion.add(
    embeddings=embeddings.tolist(),
    documents=fragmentos,
    ids=ids
)

print(f"Se han guardado {len(fragmentos)} fragmentos en ChromaDB.")

Se han guardado 3 fragmentos en ChromaDB.


In [13]:
def buscar_contexto_chroma(pregunta: str, modelo_embedding, collection, top_k=3) -> str:
    embedding = modelo_embedding.encode(pregunta).tolist()

    resultados = collection.query(query_embeddings=[embedding], n_results=top_k)

    if not resultados['documents'] or not resultados['documents'][0]:
        return None

    return "\n".join(resultados['documents'][0])

def responder_pregunta_con_together_chroma(pregunta):
    contexto = buscar_contexto_chroma(pregunta, modelo_embeddings, coleccion)

    if not contexto:
        return "No se encontró información relevante en los documentos cargados."

    prompt = f"""Contesta la siguiente pregunta usando solo la información proporcionada en el contexto.
Si no está en el contexto, responde que no tienes suficiente información.

Contexto:
{contexto}

Pregunta: {pregunta}
Respuesta:"""

    response = requests.post(
        "https://api.together.xyz/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}",
            "Content-Type": "application/json"
        },
        json={
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.3,
            "max_tokens": 512
        }
    )

    if response.status_code == 200:
        resultado = response.json()
        return resultado["choices"][0]["message"]["content"]
    else:
        return f"Error al generar respuesta: {response.status_code} - {response.text}"


In [14]:
respuesta = responder_pregunta_con_together_chroma("¿Cuál es el importe total en la factura?")
print(respuesta)


 El importe total en la factura es de 41,92 €.


In [15]:
respuesta = responder_pregunta_con_together_chroma("¿Cuál es el periodo de facturación?")
print(respuesta)

 El periodo de facturación es del 31/12/2023 a 31/01/2024 (31 días).


In [17]:
respuesta = responder_pregunta_con_together_chroma("¿Cual es el consumo eléctrico?")
print(respuesta)

 El consumo eléctrico es de 64,257 kWh (kilovatios hora) según la información proporcionada en el contexto. Este dato se encuentra en el apartado "INFORMACIÓN DEL CONSUMO ELÉCTRICO" de la factura.


In [21]:
respuesta = responder_pregunta_con_together_chroma("¿Cuál es el número de factura?")
print(respuesta)

 No tengo suficiente información para determinar el número de factura, ya que no se proporciona explícitamente en el contexto.
